In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
- Load the csv exported in Part I to a DataFrame

In [4]:
# Import data
weather_data_df = pd.read_csv('../WeatherPy/output_data/weather_df.csv')
weather_data_df


,City,Lat,Lng,Maxtemp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Airai,-8.9266,125.4092,60.26,90,91,3.13,TL,1651329814
1,Kirakira,-10.4544,161.9205,79.93,81,100,0.58,SB,1651329897
2,Port Alfred,-33.5906,26.8910,67.17,66,2,7.00,ZA,1651329657
3,Sarakhs,36.5449,61.1577,96.94,19,75,4.61,IR,1651329897
4,Vaini,-21.2000,-175.2000,77.16,94,100,10.36,TO,1651329801
...,...,...,...,...,...,...,...,...,...
562,Flin Flon,54.7682,-101.8650,40.84,75,75,4.61,CA,1651329871
563,Krasnoarmeyskoye,52.7182,50.0322,55.17,53,73,6.20,RU,1651330146
564,Rurrenabaque,-14.4413,-67.5278,81.23,65,85,3.67,BO,1651330147
565,Vestmannaeyjar,63.4427,-20.2734,47.39,76,100,16.11,IS,1651329903


 ### Humidity Heatmap

- Configure gmaps.
- Use the Lat and Lng as locations and Humidity as the weight.
- Add Heatmap layer to map.